In [1]:
import numpy
import ast 
import urllib
import scipy.optimize
import csv
import random
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import gzip
from collections import defaultdict

In [3]:
dataset = []
with open("RAW_interactions.csv", encoding="utf8") as f:
    next(f)  
    for l in f:
        fields = l.strip().split(",")  # Remove trailing newline and split by commas
        dataset.append(fields)

FileNotFoundError: [Errno 2] No such file or directory: 'RAW_interactions.csv'

In [3]:
dataset[0]
#user_id, recipe_id, date, rating, review

['38094',
 '40893',
 '2003-02-17',
 '4',
 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.']

In [4]:
recipe_ratings = {}

for row in dataset:
    # Validate that row has enough fields and fields are not empty
    if len(row) >= 4 and row[1].strip() and row[3].strip():
        recipe_id = row[1].strip()  # Clean whitespace
        rating = row[3].strip()
        
        # Additional check: Ensure 'rating' is numeric, if applicable
        if rating.isdigit():
            # Add rating to the dictionary
            if recipe_id not in recipe_ratings:
                recipe_ratings[recipe_id] = []
            recipe_ratings[recipe_id].append(int(rating))

# Example output
#for recipe_id, ratings in recipe_ratings.items():
#    print(f"Recipe ID: {recipe_id}, Ratings: {ratings}")

In [5]:
dataset_features = []
with open("RAW_recipes.csv", encoding="utf8") as f:
    next(f)  
    for l in f:
        fields = l.strip().split(",")  # Remove trailing newline and split by commas
        dataset_features.append(fields)
#name, id, minutes, contributor_id, date, tags, nutrition, number of steps, steps, description, ingredients, n_ingredinets

In [6]:
recipe_nutrition = {}
'''
with open("RAW_recipes.csv", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header

    for row in reader:
        if len(row) >= 7:  # Ensure the row has enough fields
            recipe_id = row[1].strip()  # Recipe ID (assumed second column)
            nutrition_data = row[6].strip()  # Nutrition field (assumed seventh column)

            try:
                # Safely parse the nutrition field
                nutrition_list = ast.literal_eval(nutrition_data)

                # Ensure the parsed data is a list and has at least 7 elements
                if isinstance(nutrition_list, list) and len(nutrition_list) >= 7:
                    total_fat = nutrition_list[1]  # Total fat (PDV)
                    sugar = nutrition_list[2]      # Sugar (PDV)
                    saturated_fat = nutrition_list[5]  # Saturated fat (PDV)

                    # Validate that the values are non-negative numbers
                    if all(isinstance(x, (int, float)) and x >= 0 for x in [total_fat, sugar, saturated_fat]):
                        recipe_nutrition[recipe_id] = {
                            "total_fat": total_fat,
                            "sugar": sugar,
                            "saturated_fat": saturated_fat
                        }
            except (ValueError, SyntaxError, TypeError):
                # Skip rows with invalid nutrition data
                continue
'''
with open("RAW_recipes.csv", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header

    for row in reader:
        if len(row) >= 12:  # Ensure the row has enough fields
            recipe_id = row[1].strip()  # Recipe ID (assumed second column)
            n_ingredients = row[11].strip()  # Number of ingredients field (assumed 12th column)

            # Validate and store the number of ingredients
            if n_ingredients.isdigit():  # Check if it's a valid number
                recipe_nutrition[recipe_id] = {
                    "n_ingredients": int(n_ingredients)
                }
# Example output
#for recipe_id, nutrition in recipe_nutrition.items():
#   print(f"Recipe ID: {recipe_id}, Nutrition: {nutrition}")
#extract total_fat, sugar, and saturated_fat

In [7]:
all_ids = list(recipe_nutrition.keys())
random.shuffle(all_ids)

# Calculate the split index (50/50)
split_index = len(all_ids) // 2

# Split keys into training and testing sets
train_ids = all_ids[:split_index]
test_ids = all_ids[split_index:]

# Create train and test dictionaries
train_set = {recipe_id: recipe_nutrition[recipe_id] for recipe_id in train_ids}
test_set = {recipe_id: recipe_nutrition[recipe_id] for recipe_id in test_ids}

print(f"Train set size: {len(train_set)}")
print(f"Test set size: {len(test_set)}")

Train set size: 115818
Test set size: 115819


In [8]:
def feature(datum):
    feat = [1]  # Bias term
    #feat.append(datum['total_fat'])      
    #feat.append(datum['sugar'])          
    #feat.append(datum['saturated_fat'])   
    feat.append(datum['n_ingredients'])   
    return feat

X_train = numpy.asarray([feature(train_set[recipe_id]) for recipe_id in train_set])
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

Y_train = numpy.asarray([
    numpy.mean(recipe_ratings[recipe_id]) if isinstance(recipe_ratings[recipe_id], list) else float(recipe_ratings[recipe_id])
    for recipe_id in train_set
])

model = linear_model.LinearRegression(fit_intercept=False)
model.fit(X_train, Y_train)

print("Model coefficients:", model.coef_)

Model coefficients: [ 4.37044222e+00 -2.93794302e-03]


In [9]:
X_test = numpy.asarray([feature(test_set[recipe_id]) for recipe_id in test_set])
test_recipe_ids = list(test_set.keys())
X_test_scaled = scaler.transform(X_test)

Y_test = numpy.asarray([
    numpy.mean(recipe_ratings[recipe_id]) if isinstance(recipe_ratings[recipe_id], list) else float(recipe_ratings[recipe_id])
    for recipe_id in test_recipe_ids
])

# Predict using the model
Y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse}")

def balanced_error_rate(y_true, y_pred):
    epsilon = 1e-10
    relative_errors = numpy.abs((y_true - y_pred) / y_true+epsilon)
    return numpy.mean(relative_errors)

tolerance = 0.25

# Calculate the number of correct predictions
correct_predictions = numpy.sum(numpy.abs(Y_pred - Y_test) <= tolerance)

# Calculate percent correct
percent_correct = (correct_predictions / len(Y_test)) * 100
print(f"Percent Correct: {percent_correct:.2f}%")


Mean Squared Error: 0.9798872545545332
Percent Correct: 14.47%


In [9]:
#implementation of similarity/pop
import csv
from collections import defaultdict

#THIS HOLDS EVERYTHING
# Load data function
def load_data(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            user_id, recipe_id, rating = row[0], row[1], float(row[3])
            data.append((user_id, recipe_id, rating))
    return data

# Load train, validation, and test data
ratingsTrain = load_data("interactions_train.csv")
ratingsValid = load_data("interactions_validation.csv")
ratingsTest = load_data("interactions_test.csv")

print(f"Training: {len(ratingsTrain)}, Validation: {len(ratingsValid)}, Test: {len(ratingsTest)}")

Training: 698901, Validation: 7023, Test: 12455


In [14]:
ratingsPerUser = defaultdict(list)
ratingsPerRecipe = defaultdict(list)
 
for u, b, r in ratingsTrain:
    ratingsPerUser[u].append((b, r))
    ratingsPerRecipe[b].append((u, r))


#any interaction/any rating was added
recipeCount = defaultdict(int)

for _, recipe, _ in ratingsTrain:
    recipeCount[recipe] += 1


mostPopular = list(recipeCount.items())

# mostPopular = sorted(recipeCount.items(), key=lambda x: -x[1])
mostPopular.sort(key=lambda x: x[1])
mostPopular.reverse()


print(mostPopular[:10])
returnSet = set()
count = 0
totalInteractions = len(ratingsTrain)

for recipeID, interactionCount in mostPopular:
    count += interactionCount
    returnSet.add(recipeID)
    if count > totalInteractions / 2:  # Threshold for popularity
        break

print(f"Number of popular recipes: {len(returnSet)}")

[('27208', 1091), ('89204', 1075), ('32204', 897), ('39087', 894), ('69173', 787), ('54257', 758), ('22782', 738), ('68955', 681), ('25885', 677), ('28148', 666)]
Number of popular recipes: 14382


In [16]:
def Jaccard(s1, s2):
    intersection = len(s1 & s2)
    union = len(s1 | s2)
    return intersection / union if union > 0 else 0

rating_threshold = 4
correct=0
threshold_jaccard=0.01

for user, recipe, actual_rating in ratingsValid:  # Iterate through the validation set
    # Popularity prediction
    popular_pred = recipe in returnSet

    # Jaccard prediction
    max_sim = 0
    for read_recipe,_ in ratingsPerUser[user]:
        users_for_recipe = set([u for u, _ in ratingsPerRecipe[recipe]])
        users_for_read_recipe = set([u for u, _ in ratingsPerRecipe[read_recipe]])
        sim = Jaccard(users_for_recipe, users_for_read_recipe)
        if sim>0: print(sim)
        max_sim = max(max_sim, sim)
    jaccard_pred = max_sim > threshold_jaccard

    # Hybrid prediction: A recipe is well-rated if it meets either popularity or Jaccard criteria
    hybrid_pred = popular_pred and jaccard_pred

    # Convert actual rating to a binary label: well-rated (1) or not well-rated (0)
    actual = 1 if actual_rating >= rating_threshold else 0

    # Check if the hybrid prediction matches the actual label
    if hybrid_pred == actual:
        correct += 1

# Calculate accuracy
accuracy = correct / len(ratingsValid)
print(f"Hybrid model accuracy for well-rated prediction: {accuracy}")

Hybrid model accuracy for well-rated prediction: 0.15990317528121886
